<a href="https://colab.research.google.com/github/mostafa-ja/contextAware/blob/main/context_aware_engine_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# PART 1 — CONFIG + FEATURE SET (UPDATED WITH HOLIDAY SUPPORT)
# ============================================================

import json
import urllib.request
import math
import os
import datetime
import time
import random

# ============================================================
# 0. CONFIGURATION
# ============================================================

LATITUDE = 38.0792
LONGITUDE = 46.2887

DATA_DIR = "./dataset"
FRESHNESS_STATE_FILE = "freshness_state.json"

FRESHNESS_DECAY_FACTOR = 0.9
FRESHNESS_MIN_WEIGHT = 0.05
FRESHNESS_MAX_WEIGHT = 1.0
DEFAULT_RECOVERY_RATE = 0.01

# Subcategory-specific recovery modifiers
RECOVERY_RATE_BY_SUBCATEGORY = {
    "warming_drinks": 0.005,
    "cooling_drinks": 0.005,
    "comfort_meal": 0.005,
    "light_meal": 0.005,
    "quick_snack": 0.005,
    "seasonal_special": 0.005,

    "outdoor_active": 0.01,
    "outdoor_shade": 0.01,
    "indoor_cozy": 0.01,
    "home_micro": 0.01,
    "fitness_wellness": 0.01,
    "travel_errand": 0.01,
    "social_community": 0.01,

    "layering_core": 0.008,
    "accessories_weather": 0.008,
    "footwear": 0.008,
    "protection_items": 0.008,
    "comfort_recovery": 0.01,
    "home_comfort": 0.01,

    "movies": 0.003,
    "books": 0.003,
    "music": 0.002,
    "creative": 0.005,
    "boosters": 0.01,
    "relaxation": 0.01,
    "energizing": 0.01,
    "mental_health_support": 0.01,
    "productivity_focus": 0.01,
}

# Iran-specific: weekend = Friday
WEEKEND_DAYS = {4}

# Group weights (unchanged)
GROUP_WEIGHTS = {
    "temp": 1.0,
    "weather": 0.9,
    "social": 0.6,
    "mood": 0.6,
    "time": 1.0,
    "location": 1.0,
    "events": 0.8,
    "humidity": 0.6,
    "wind": 0.6,
    "season": 0.5,
    "energy": 0.5
}

MIN_SCORE_THRESHOLD = -1.0

# ============================================================
# MASTER FEATURE LIST — UNCHANGED (Dataset Compatibility)
# ============================================================

FEATURE_NAMES = [
    # Temperature
    "temp_extreme_cold", "temp_cold", "temp_cool", "temp_warm", "temp_hot",

    # Weather
    "weather_clear", "weather_partly_cloudy", "weather_cloudy", "weather_fog",
    "weather_drizzle", "weather_rain", "weather_rain_shower", "weather_snow",
    "weather_snow_shower", "weather_thunderstorm",

    # Humidity
    "humidity_very_dry", "humidity_dry", "humidity_comfortable",
    "humidity_humid", "humidity_very_humid",

    # Wind
    "wind_calm", "wind_breeze", "wind_windy", "wind_strong", "wind_storm",

    # Time
    "time_late_night", "time_early_morning", "time_morning",
    "time_afternoon", "time_evening", "time_night",

    # Day Type
    "day_weekend", "day_holiday", "day_holiday_eve", "day_workday",

    # Season
    "season_spring", "season_summer", "season_autumn", "season_winter",

    # Events
    "romantic_event", "national_festival", "national_mourning", "cultural_tradition",

    # Social
    "social_solo", "social_couple", "social_family", "social_friends", "social_group",

    # Mood
    "mood_calm", "mood_energetic", "mood_happy", "mood_sad", "mood_thoughtful",
    "mood_romantic", "mood_nostalgic", "mood_stressed", "mood_relaxed",

    # Location
    "location_indoor", "location_outdoor", "location_home",

    # Energy
    "energy_very_low", "energy_low", "energy_medium",
    "energy_high", "energy_very_high"
]


# ============================================================
# EVENT NORMALIZATION HELPERS
# ============================================================

def _normalize_event_key(name: str) -> str:
    if not isinstance(name, str):
        return ""
    s = name.strip().lower()
    for ch in [" ", "‌", "_", "-", "ـ"]:
        s = s.replace(ch, "")
    return s


EVENT_CANONICAL_MAP = {
    # Norooz
    "نوروز": "norooz", "nowruz": "norooz", "norooz": "norooz",

    # Chaharshanbe Suri
    "چهارشنبهسوری": "chaharshanbe_suri", "chaharshanbesuri": "chaharshanbe_suri",

    # Yalda
    "یلدا": "yalda", "yalda": "yalda",

    # Sizdah Bedar
    "سیزدهبدر": "sizdah_bedar", "sizdahbedar": "sizdah_bedar",

    # Mourning
    "عاشورا": "ashura", "ashura": "ashura",
    "تاسوعا": "tasua", "tasua": "tasua",
    "اربعین": "arbaeen", "arbaeen": "arbaeen",

    # Eids
    "عیدفطر": "eid_fitr", "eidfitr": "eid_fitr",
    "عیدقربان": "eid_adha", "eidadha": "eid_adha",
    "عیدمبعث": "eid_mabath", "eidmabath": "eid_mabath",
    "نیمهشعبان": "mid_shaban", "midshaban": "mid_shaban",

    # Family days
    "روزمادر": "mother_day", "roozemadar": "mother_day",
    "روزپدر": "father_day", "roozpedar": "father_day",
    "روززن": "women_day",
    "روزمرد": "men_day", "roozemard": "men_day",

    # Valentine
    "ولنتاین": "valentine", "valentine": "valentine",
}

# ============================================================
# 1. FUZZY LOGIC CORE
# ============================================================

def fuzzy_triangular(value: float, center: float, width: float) -> float:
    distance = abs(value - center)
    return max(0.0, min(1.0, 1.0 - distance / width))


def fuzzy_gaussian(value: float, center: float, sigma: float) -> float:
    return math.exp(-0.5 * ((value - center) / sigma) ** 2)


def fuzzy_trapezoidal(value: float, a: float, b: float, c: float, d: float) -> float:
    if value <= a or value >= d:
        return 0.0
    if b <= value <= c:
        return 1.0
    if a < value < b:
        return (value - a) / (b - a)
    if c < value < d:
        return (d - value) / (d - c)
    return 0.0


FUZZY_CONFIG = {
    # Temperature  (norm = (T + 10) / 50)
    "temp_extreme_cold": {"type": "gaussian",   "center": 0.05, "sigma": 0.055},
    "temp_cold":         {"type": "triangular", "center": 0.20, "width": 0.20},
    "temp_cool":         {"type": "trapezoidal","a": 0.28, "b": 0.38, "c": 0.52, "d": 0.64},
    "temp_warm":         {"type": "triangular", "center": 0.70, "width": 0.22},
    "temp_hot":          {"type": "gaussian",   "center": 0.90, "sigma": 0.085},

    # Humidity (norm = RH% / 100)
    "humidity_very_dry":    {"type": "triangular", "center": 0.10, "width": 0.22},
    "humidity_dry":         {"type": "triangular", "center": 0.25, "width": 0.22},
    "humidity_comfortable": {"type": "trapezoidal","a": 0.37, "b": 0.43, "c": 0.55, "d": 0.63},
    "humidity_humid":       {"type": "triangular", "center": 0.70, "width": 0.22},
    "humidity_very_humid":  {"type": "gaussian",   "center": 0.88, "sigma": 0.10},

    # Wind (norm = speed_kmh / 60)
    "wind_calm":   {"type": "triangular", "center": 0.05, "width": 0.22},
    "wind_breeze": {"type": "triangular", "center": 0.20, "width": 0.22},
    "wind_windy":  {"type": "triangular", "center": 0.45, "width": 0.22},
    "wind_strong": {"type": "triangular", "center": 0.70, "width": 0.22},
    "wind_storm":  {"type": "gaussian",   "center": 0.92, "sigma": 0.08},

    # Time-of-day (hour in [0, 24))
    "time_late_night": {
        "type": "triangular",
        "center": 1.5,   # ~23:00–04:00 wrap-around
        "width": 5.0,
    },
    "time_early_morning": {
        "type": "triangular",
        "center": 6.0,   # ~04:00–08:00
        "width": 2.0,
    },
    "time_morning": {
        "type": "triangular",
        "center": 9.25,  # ~07:00–11:30
        "width": 2.25,
    },
    "time_afternoon": {
        "type": "triangular",
        "center": 15.0,  # ~12:30–17:30
        "width": 2.5,
    },
    "time_evening": {
        "type": "triangular",
        "center": 19.0,  # ~17:30–20:30
        "width": 1.5,
    },
    "time_night": {
        "type": "triangular",
        "center": 21.5,  # ~19:00–24:00
        "width": 2.5,
    },
}


def apply_fuzzy(feature_name: str, value: float) -> float:
    cfg = FUZZY_CONFIG.get(feature_name)
    if not cfg:
        return 0.0
    t = cfg["type"]
    if t == "triangular":
        return fuzzy_triangular(value, cfg["center"], cfg["width"])
    if t == "gaussian":
        return fuzzy_gaussian(value, cfg["center"], cfg["sigma"])
    if t == "trapezoidal":
        return fuzzy_trapezoidal(value, cfg["a"], cfg["b"], cfg["c"], cfg["d"])
    return 0.0


# ============================================================
# 2. WEATHER VECTORIZER
# ============================================================

class WeatherVectorizer:
    @staticmethod
    def vectorize_code(code: int) -> dict:
        f = {k: 0.0 for k in FEATURE_NAMES if k.startswith("weather_")}

        if code == 0:
            f["weather_clear"] = 1.0
        elif code == 1:
            f["weather_clear"] = 0.8
            f["weather_partly_cloudy"] = 0.2
        elif code == 2:
            f["weather_partly_cloudy"] = 1.0
        elif code == 3:
            f["weather_cloudy"] = 1.0
        elif code in [45, 48]:
            f["weather_fog"] = 1.0
        elif code in [51, 56]:
            f["weather_drizzle"] = 0.6
        elif code == 53:
            f["weather_drizzle"] = 0.8
        elif code in [55, 57]:
            f["weather_drizzle"] = 1.0
        elif code in [61, 66]:
            f["weather_rain"] = 0.6
        elif code == 63:
            f["weather_rain"] = 0.8
        elif code in [65, 67]:
            f["weather_rain"] = 1.0
        elif code == 71:
            f["weather_snow"] = 0.6
        elif code == 73:
            f["weather_snow"] = 0.8
        elif code in [75, 77]:
            f["weather_snow"] = 1.0
        elif code == 80:
            f["weather_rain_shower"] = 0.6
        elif code == 81:
            f["weather_rain_shower"] = 0.8
        elif code == 82:
            f["weather_rain_shower"] = 1.0
        elif code == 85:
            f["weather_snow_shower"] = 0.7
        elif code == 86:
            f["weather_snow_shower"] = 1.0
        elif code in [95, 96, 99]:
            f["weather_thunderstorm"] = 1.0
        else:
            f["weather_clear"] = 0.5

        return f

    @staticmethod
    def vectorize_temp(temp_c: float, feels_like_c: float) -> dict:
        effective = 0.7 * feels_like_c + 0.3 * temp_c
        norm = max(0.0, min(1.0, (effective + 10.0) / 50.0))
        return {
            "temp_extreme_cold": apply_fuzzy("temp_extreme_cold", norm),
            "temp_cold":         apply_fuzzy("temp_cold", norm),
            "temp_cool":         apply_fuzzy("temp_cool", norm),
            "temp_warm":         apply_fuzzy("temp_warm", norm),
            "temp_hot":          apply_fuzzy("temp_hot", norm),
        }

    @staticmethod
    def vectorize_humidity(humidity_percent: float) -> dict:
        norm = humidity_percent / 100.0
        return {
            "humidity_very_dry":    apply_fuzzy("humidity_very_dry", norm),
            "humidity_dry":         apply_fuzzy("humidity_dry", norm),
            "humidity_comfortable": apply_fuzzy("humidity_comfortable", norm),
            "humidity_humid":       apply_fuzzy("humidity_humid", norm),
            "humidity_very_humid":  apply_fuzzy("humidity_very_humid", norm),
        }

    @staticmethod
    def vectorize_wind(speed_kmh: float) -> dict:
        norm = min(1.0, speed_kmh / 60.0)
        return {
            "wind_calm":   apply_fuzzy("wind_calm", norm),
            "wind_breeze": apply_fuzzy("wind_breeze", norm),
            "wind_windy":  apply_fuzzy("wind_windy", norm),
            "wind_strong": apply_fuzzy("wind_strong", norm),
            "wind_storm":  apply_fuzzy("wind_storm", norm),
        }


# ============================================================
# 3. TIME & PERSIAN SEASON VECTORIZERS
# ============================================================

class TimeVectorizer:
    @staticmethod
    def vectorize(hour: float, sunrise_hour: float, sunset_hour: float, weekday: int) -> dict:
        is_day = 1 if sunrise_hour <= hour <= sunset_hour else 0

        result = {
            name: apply_fuzzy(name, hour)
            for name in [
                "time_late_night",
                "time_early_morning",
                "time_morning",
                "time_afternoon",
                "time_evening",
                "time_night",
            ]
        }

        if not is_day:
            result["time_evening"] *= 1.15
            result["time_night"] *= 1.25
            result["time_late_night"] *= 1.10
            result["time_morning"] *= 0.70

        if weekday == 3:  # Thursday night boost
            result["time_evening"] *= 1.15
            result["time_night"] *= 1.20

        for k in result:
            result[k] = max(0.0, min(1.0, result[k]))
        return result


def vectorize_season(persian_month: int, persian_day: int) -> dict:
    """
    Fuzzy seasons based on the Persian (Jalali) calendar.

    Rough mapping:
      1–3   → Spring
      4–6   → Summer
      7–9   → Autumn
      10–12 → Winter
    """
    try:
        m = int(persian_month)
        d = int(persian_day)
    except (TypeError, ValueError):
        m, d = 1, 1

    m = max(1, min(12, m))
    d = max(1, min(30, d))

    # Continuous position in [1, 12], roughly mid-month
    x = m + (d - 15) / 30.0
    x = max(1.0, min(12.0, x))

    def gaussian_circular(x_val: float, center: float, width: float) -> float:
        diff = abs(x_val - center)
        diff = min(diff, 12.0 - diff)
        return math.exp(-0.5 * (diff / width) ** 2)

    # Centers:
    #  2 → Spring, 5 → Summer, 8 → Autumn, 11 → Winter
    spring = gaussian_circular(x, 2.0, 1.3)
    summer = gaussian_circular(x, 5.0, 1.3)
    autumn = gaussian_circular(x, 8.0, 1.3)
    winter = gaussian_circular(x, 11.0, 1.3)

    total = spring + summer + autumn + winter
    if total > 0:
        spring /= total
        summer /= total
        autumn /= total
        winter /= total

    return {
        "season_spring": spring,
        "season_summer": summer,
        "season_autumn": autumn,
        "season_winter": winter,
    }




# ============================================================
# 4. CONTEXT BUILDER — WEATHER + PERSIAN DAY + EVENTS
# ============================================================

class ContextBuilder:
    def build(self, weather_data: dict, sunrise_hour: float, sunset_hour: float, day_info: dict) -> dict:
        """
        day_info example:
        {
            "day_type": "day_workday",  # or "day_weekend" / "day_holiday" / "day_holiday_eve"
            "persian_month": 9,
            "persian_day": 30,
            "is_holiday": False,
            "events": ["یلدا", "valentine"]
        }
        """
        current = weather_data["current"]

        # 1) WEATHER FEATURES
        temp_f = WeatherVectorizer.vectorize_temp(
            current["temperature_2m"],
            current["apparent_temperature"],
        )
        weather_f = WeatherVectorizer.vectorize_code(current["weather_code"])
        humidity_f = WeatherVectorizer.vectorize_humidity(current["relative_humidity_2m"])
        wind_f = WeatherVectorizer.vectorize_wind(current["wind_speed_10m"])

        # 2) TIME FEATURES (Gregorian-based)
        now = datetime.datetime.now()
        hour = now.hour + now.minute / 60.0
        weekday = now.weekday()
        time_f = TimeVectorizer.vectorize(hour, sunrise_hour, sunset_hour, weekday)

        # 3) PERSIAN DATE + DAY TYPE
        persian_month = day_info.get("persian_month", 1)
        persian_day   = day_info.get("persian_day", 1)
        is_holiday    = bool(day_info.get("is_holiday", False))
        day_type      = day_info.get("day_type", "day_workday")

        # 4) SEASON (Persian-based)
        season_f = vectorize_season(persian_month, persian_day)

        # 5) DAY-TYPE FEATURES (DRIVEN BY INPUT)
        #    We trust day_type, but if it's invalid we fall back to weekday.
        day_f = {
            "day_weekend": 0.0,
            "day_holiday": 0.0,
            "day_holiday_eve": 0.0,
            "day_workday": 0.0,
        }

        if day_type in day_f:
            day_f[day_type] = 1.0
        else:
            # Fallback: derive from Gregorian weekday
            is_weekend_py = 1.0 if weekday in WEEKEND_DAYS else 0.0
            day_f["day_weekend"] = is_weekend_py
            day_f["day_workday"] = 1.0 - is_weekend_py

        # Explicit holiday flag strengthens holiday-ness even if day_type mis-set.
        if is_holiday:
            # Boost holiday feature
            day_f["day_holiday"] = max(day_f["day_holiday"], 1.0)
            # If it was marked workday, soften that
            day_f["day_workday"] *= 0.4

        # 6) EVENT FEATURES (initial)
        event_f = {
            "romantic_event": 0.0,
            "national_festival": 0.0,
            "national_mourning": 0.0,
            "cultural_tradition": 0.0,
        }

        # 7) BASE INFERENCES (from weather + time only)
        mood_f = self.infer_mood(weather_f, time_f)

        # weekend-like behavior: holiday or weekend both count
        is_weekend_like = (
            day_f["day_weekend"] > 0.5 or
            day_f["day_holiday"] > 0.5 or
            day_type == "day_holiday_eve"
        )
        social_f = self.infer_social(time_f, 1.0 if is_weekend_like else 0.0)

        location_f = self.infer_location(weather_f, wind_f, temp_f, time_f)
        energy_f   = self.infer_energy(time_f)

        # 8) ENRICH WITH PERSIAN DAY INFO (HOLIDAY + EVENTS)
        self.enrich_with_persian_day_info(
            day_info=day_info,
            season_f=season_f,
            day_f=day_f,
            event_f=event_f,
            mood_f=mood_f,
            social_f=social_f,
            location_f=location_f,
            energy_f=energy_f,
        )

        # 9) MERGE CONTEXT
        ctx = {}
        ctx.update(temp_f)
        ctx.update(weather_f)
        ctx.update(humidity_f)
        ctx.update(wind_f)
        ctx.update(time_f)
        ctx.update(day_f)
        ctx.update(season_f)
        ctx.update(event_f)
        ctx.update(mood_f)
        ctx.update(social_f)
        ctx.update(location_f)
        ctx.update(energy_f)
        return ctx

    # --------------------------------------------------------
    # INFERRED GROUPS (BASELINES, before holiday/events)
    # --------------------------------------------------------

    def infer_mood(self, w: dict, t: dict) -> dict:
        mood = {k: 0.1 for k in FEATURE_NAMES if k.startswith("mood_")}
        mood["mood_calm"] = 0.2
        mood["mood_happy"] = 0.3

        if w["weather_rain"] > 0.5:
            mood["mood_calm"] += 0.3
            mood["mood_thoughtful"] += 0.3

        if w["weather_clear"] > 0.7:
            mood["mood_happy"] += 0.4
            mood["mood_energetic"] += 0.3

        if t["time_late_night"] > 0.5:
            mood["mood_calm"] += 0.4
            mood["mood_thoughtful"] += 0.3

        return {k: max(0.0, min(1.0, v)) for k, v in mood.items()}

    def infer_social(self, t: dict, is_weekend_like: float) -> dict:
        social = {k: 0.1 for k in FEATURE_NAMES if k.startswith("social_")}
        social["social_solo"] = 0.3

        if is_weekend_like > 0.5:
            social["social_family"] += 0.4
            social["social_friends"] += 0.3

        if t["time_evening"] > 0.5:
            social["social_family"] += 0.2

        return {k: max(0.0, min(1.0, v)) for k, v in social.items()}

    def infer_location(self, w: dict, wind: dict, temp: dict, time_f: dict) -> dict:
        loc = {
            "location_indoor": 0.4,
            "location_outdoor": 0.4,
            "location_home": 0.3,
        }

        rain_strength = w["weather_rain"] + 0.7 * w["weather_rain_shower"] + 0.4 * w["weather_drizzle"]
        snow_strength = w["weather_snow"] + 0.6 * w["weather_snow_shower"]
        thunder_strength = w["weather_thunderstorm"]
        bad_weather = max(rain_strength, snow_strength, thunder_strength)

        if bad_weather > 0.2:
            boost = min(0.6, bad_weather)
            loc["location_indoor"] += 0.4 * boost
            loc["location_outdoor"] -= 0.5 * boost
            loc["location_home"] += 0.3 * boost

        wind_bad = (
            0.4 * wind["wind_windy"] +
            0.6 * wind["wind_strong"] +
            1.0 * wind["wind_storm"]
        )
        if wind_bad > 0.25:
            loc["location_outdoor"] -= 0.4 * wind_bad
            loc["location_indoor"] += 0.2 * wind_bad

        too_cold = temp["temp_extreme_cold"] + 0.7 * temp["temp_cold"]
        too_hot = temp["temp_hot"]

        if too_cold > 0.2:
            loc["location_indoor"] += 0.3 * too_cold
            loc["location_outdoor"] -= 0.4 * too_cold

        if too_hot > 0.2:
            loc["location_indoor"] += 0.3 * too_hot
            loc["location_outdoor"] -= 0.3 * too_hot

        perfect_weather = w["weather_clear"] * wind["wind_calm"]
        if perfect_weather > 0.5:
            loc["location_outdoor"] += 0.35 * perfect_weather
            loc["location_indoor"] -= 0.20 * perfect_weather

        if time_f.get("time_night", 0.0) > 0.4:
            loc["location_outdoor"] -= 0.25 * time_f["time_night"]
            loc["location_home"] += 0.2 * time_f["time_night"]

        if time_f.get("time_late_night", 0.0) > 0.3:
            loc["location_outdoor"] -= 0.35 * time_f["time_late_night"]
            loc["location_home"] += 0.3 * time_f["time_late_night"]

        # Normalize to sum=1
        for k in loc:
            loc[k] = max(0.0, min(1.0, loc[k]))
        total = sum(loc.values()) or 1.0
        for k in loc:
            loc[k] /= total

        return loc

    def infer_energy(self, t: dict) -> dict:
        e = {k: 0.2 for k in FEATURE_NAMES if k.startswith("energy_")}
        if t["time_morning"] > 0.5:
            e["energy_high"] += 0.4
        if t["time_late_night"] > 0.5:
            e["energy_very_low"] += 0.6
        return {k: max(0.0, min(1.0, v)) for k, v in e.items()}





    # --------------------------------------------------------
    # ENRICH WITH HOLIDAY + EVENTS (PERSIAN)
    # --------------------------------------------------------

    def enrich_with_persian_day_info(
        self,
        day_info: dict,
        season_f: dict,
        day_f: dict,
        event_f: dict,
        mood_f: dict,
        social_f: dict,
        location_f: dict,
        energy_f: dict,
    ) -> None:
        def clamp01(x): return max(0.0, min(1.0, x))

        # -------- Holiday baseline influence --------
        if day_info.get("is_holiday"):
            # Treat strongly as weekend-like
            day_f["day_holiday"] = clamp01(max(day_f.get("day_holiday", 0.0), 1.0))
            # Holiday → more family/social mood
            mood_f["mood_happy"] = clamp01(mood_f.get("mood_happy", 0.0) + 0.2)
            mood_f["mood_relaxed"] = clamp01(mood_f.get("mood_relaxed", 0.0) + 0.2)

            social_f["social_family"] = clamp01(social_f.get("social_family", 0.0) + 0.4)
            social_f["social_group"] = clamp01(social_f.get("social_group", 0.0) + 0.3)

            location_f["location_home"] = clamp01(location_f.get("location_home", 0.0) + 0.2)

        # -------- Normalize events --------
        events_raw = day_info.get("events") or []
        normalized_events = set()
        for e in events_raw:
            key = _normalize_event_key(str(e))
            canon = EVENT_CANONICAL_MAP.get(key)
            if canon:
                normalized_events.add(canon)

        # Helpers
        def add_mood(k, d):    mood_f[k] = clamp01(mood_f.get(k, 0.0) + d)
        def add_social(k, d):  social_f[k] = clamp01(social_f.get(k, 0.0) + d)
        def add_loc(k, d):     location_f[k] = clamp01(location_f.get(k, 0.0) + d)
        def add_energy(k, d):  energy_f[k] = clamp01(energy_f.get(k, 0.0) + d)

        # -------- Yalda --------
        if "yalda" in normalized_events:
            event_f["cultural_tradition"] = 1.0

            add_mood("mood_nostalgic", 0.5)
            add_mood("mood_romantic", 0.3)
            add_mood("mood_relaxed", 0.3)

            add_social("social_family", 0.5)
            add_social("social_friends", 0.3)

            add_loc("location_home", 0.5)
            add_loc("location_indoor", 0.3)

            season_f["season_winter"] = max(season_f.get("season_winter", 0.0), 0.7)

        # -------- Norooz --------
        if "norooz" in normalized_events:
            event_f["national_festival"] = 1.0
            event_f["cultural_tradition"] = max(event_f["cultural_tradition"], 0.9)

            add_mood("mood_happy", 0.4)
            add_mood("mood_energetic", 0.3)

            add_social("social_family", 0.5)
            add_social("social_friends", 0.4)
            add_social("social_group", 0.3)

            add_loc("location_outdoor", 0.3)
            add_loc("location_home", 0.2)

            add_energy("energy_high", 0.3)

        # -------- Sizdah Bedar --------
        if "sizdah_bedar" in normalized_events:
            event_f["national_festival"] = max(event_f["national_festival"], 0.9)
            event_f["cultural_tradition"] = max(event_f["cultural_tradition"], 0.9)

            add_social("social_family", 0.4)
            add_social("social_group", 0.4)

            add_loc("location_outdoor", 0.6)
            add_loc("location_home", -0.2)

            add_mood("mood_happy", 0.4)
            add_mood("mood_energetic", 0.4)

            add_energy("energy_high", 0.3)

        # -------- Mourning Days --------
        if any(e in normalized_events for e in ["ashura", "tasua", "arbaeen"]):
            event_f["national_mourning"] = 1.0

            add_mood("mood_sad", 0.6)
            add_mood("mood_thoughtful", 0.5)
            mood_f["mood_happy"] = clamp01(mood_f.get("mood_happy", 0.0) * 0.6)
            mood_f["mood_energetic"] = clamp01(mood_f.get("mood_energetic", 0.0) * 0.7)

            add_social("social_group", 0.4)
            add_social("social_family", 0.3)

            add_loc("location_outdoor", 0.3)
            add_loc("location_home", 0.1)

            add_energy("energy_low", 0.3)

        # -------- Eids --------
        if any(e in normalized_events for e in ["eid_fitr", "eid_adha", "eid_mabath", "mid_shaban"]):
            event_f["national_festival"] = max(event_f["national_festival"], 0.9)
            event_f["cultural_tradition"] = max(event_f["cultural_tradition"], 0.7)

            add_mood("mood_happy", 0.4)
            add_mood("mood_relaxed", 0.2)

            add_social("social_family", 0.4)
            add_social("social_friends", 0.3)
            add_social("social_group", 0.3)

            add_loc("location_home", 0.2)
            add_loc("location_outdoor", 0.2)

            add_energy("energy_medium", 0.2)

        # -------- Family Days --------
        if any(e in normalized_events for e in ["mother_day", "father_day", "women_day", "men_day"]):
            add_mood("mood_nostalgic", 0.3)
            add_mood("mood_happy", 0.3)
            add_social("social_family", 0.5)
            add_loc("location_home", 0.3)
            event_f["cultural_tradition"] = max(event_f["cultural_tradition"], 0.4)

        # -------- Valentine --------
        if "valentine" in normalized_events:
            event_f["romantic_event"] = 1.0

            add_mood("mood_romantic", 0.6)
            add_mood("mood_happy", 0.3)

            add_social("social_couple", 0.7)
            add_social("social_friends", 0.2)

            add_loc("location_home", 0.3)
            add_loc("location_outdoor", 0.2)

            add_energy("energy_medium", 0.2)

        # Final clamp
        for d in (season_f, day_f, event_f, mood_f, social_f, location_f, energy_f):
            for k, v in d.items():
                d[k] = clamp01(v)


# ============================================================
# 5. SUGGESTION VALIDATOR (UNCHANGED)
# ============================================================

def validate_suggestion(item: dict, filename: str, index: int) -> list:
    errors = []
    for key in ("text", "category", "subcategory", "preferencesJson"):
        if key not in item:
            errors.append(f"[{filename}#{index}] Missing field '{key}'")

    prefs = item.get("preferencesJson", {})
    if not isinstance(prefs, dict):
        errors.append(f"[{filename}#{index}] preferencesJson must be an object")
        return errors

    for feat, val in prefs.items():
        if feat not in FEATURE_NAMES:
            errors.append(f"[{filename}#{index}] Unknown feature '{feat}'")
        if not isinstance(val, (int, float)):
            errors.append(f"[{filename}#{index}] Non-numeric value for '{feat}': {val}")
        elif val < -10.0 or val > 1.0:
            errors.append(f"[{filename}#{index}] Value {val} out of range [-10, 1] for '{feat}'")

    if 0 < len(prefs) < 10:
        errors.append(f"[{filename}#{index}] Too few preferences ({len(prefs)}); expected ~15–25")

    return errors


# ============================================================
# FEATURE GROUP LOOKUP
# ============================================================

FEATURE_GROUP = {}
for feat in FEATURE_NAMES:
    for group in GROUP_WEIGHTS.keys():
        if feat.startswith(group + "_"):
            FEATURE_GROUP[feat] = group
            break


# ============================================================
# 6. SUGGESTION ENGINE — SCORING + FRESHNESS
# ============================================================

class SuggestionEngine:
    def __init__(self):
        self.suggestions = []
        self.freshness_state = self._load_freshness_state()

    # --------------------------------------------------------
    # FRESHNESS SYSTEM
    # --------------------------------------------------------

    def _item_id(self, item: dict) -> str:
        """Returns stable ID"""
        if "id" in item:
            return str(item["id"])
        return "text:" + item.get("text", "")

    def _load_freshness_state(self):
        if not os.path.exists(FRESHNESS_STATE_FILE):
            return {}
        try:
            with open(FRESHNESS_STATE_FILE, "r", encoding="utf-8") as f:
                return json.load(f)
        except:
            return {}

    def save_freshness_state(self):
        try:
            with open(FRESHNESS_STATE_FILE, "w", encoding="utf-8") as f:
                json.dump(self.freshness_state, f, ensure_ascii=False, indent=2)
        except Exception as e:
            print(f"⚠️ Failed to save freshness state: {e}")

    def _ensure_item_state(self, item: dict):
        sub = item.get("subcategory", "")
        iid = self._item_id(item)
        if iid not in self.freshness_state:
            self.freshness_state[iid] = {
                "weight": FRESHNESS_MAX_WEIGHT,
                "subcategory": sub,
            }

    def _get_recovery_rate(self, subcat: str) -> float:
        return RECOVERY_RATE_BY_SUBCATEGORY.get(subcat, DEFAULT_RECOVERY_RATE)

    def refresh_all_before_run(self):
        """Runs once per session.
        Slowly restores freshness for all items."""
        for item in self.suggestions:
            sub = item.get("subcategory", "")
            iid = self._item_id(item)
            state = self.freshness_state.setdefault(
                iid,
                {"weight": FRESHNESS_MAX_WEIGHT, "subcategory": sub},
            )

            w = state.get("weight", FRESHNESS_MAX_WEIGHT)
            w += self._get_recovery_rate(sub)
            w = min(FRESHNESS_MAX_WEIGHT, max(FRESHNESS_MIN_WEIGHT, w))

            state["weight"] = w
            state["subcategory"] = sub

    def get_item_freshness(self, item: dict) -> float:
        iid = self._item_id(item)
        st = self.freshness_state.get(iid)
        if not st:
            return FRESHNESS_MAX_WEIGHT
        w = st.get("weight", FRESHNESS_MAX_WEIGHT)
        return min(FRESHNESS_MAX_WEIGHT, max(FRESHNESS_MIN_WEIGHT, w))

    def apply_decay_for_items(self, items: list):
        """Decay applied after selecting main + sidekicks."""
        for item in items:
            sub = item.get("subcategory", "")
            iid = self._item_id(item)
            state = self.freshness_state.setdefault(
                iid,
                {"weight": FRESHNESS_MAX_WEIGHT, "subcategory": sub},
            )

            w = state.get("weight", FRESHNESS_MAX_WEIGHT)
            w *= FRESHNESS_DECAY_FACTOR
            w = min(FRESHNESS_MAX_WEIGHT, max(FRESHNESS_MIN_WEIGHT, w))

            state["weight"] = w
            state["subcategory"] = sub

    # --------------------------------------------------------
    # LOADING DATA
    # --------------------------------------------------------

    def load_data(self, root_dir: str):
        print(f"📂 Loading dataset from {root_dir}")
        count_valid = 0
        count_total = 0

        for r, _, files in os.walk(root_dir):
            for file in files:
                if not file.endswith(".json"):
                    continue

                path = os.path.join(r, file)
                try:
                    with open(path, "r", encoding="utf-8") as f:
                        data = json.load(f)
                except Exception as e:
                    print(f"❌ Failed to read {file}: {e}")
                    continue

                if not isinstance(data, list):
                    print(f"⚠️ {file} does not contain a JSON array.")
                    continue

                for i, item in enumerate(data):
                    count_total += 1
                    errs = validate_suggestion(item, file, i)
                    if errs:
                        for e in errs:
                            print("   ", e)
                        continue

                    self.suggestions.append(item)
                    self._ensure_item_state(item)
                    count_valid += 1

        print(f"✅ Loaded {count_valid} valid suggestions (from {count_total} total).")

    # --------------------------------------------------------
    # SCORING
    # --------------------------------------------------------

    def _is_veto(self, ctx: dict, prefs: dict) -> bool:
        """If any feature has preference <= -9 and context>0.1 → veto."""
        for feat, pref in prefs.items():
            if pref <= -9.0 and ctx.get(feat, 0.0) > 0.1:
                return True
        return False

    def _score_single(self, ctx: dict, prefs: dict) -> float:
        total = 0.0
        for feat, pref_val in prefs.items():
            ctx_val = ctx.get(feat, 0.0)
            if ctx_val == 0:
                continue
            group = FEATURE_GROUP.get(feat)
            gw = GROUP_WEIGHTS.get(group, 0.5)
            total += gw * (pref_val * ctx_val)
        return total

    def calculate_item_score(self, item: dict, ctx: dict) -> float:
        prefs = item.get("preferencesJson", {})

        # veto first
        if self._is_veto(ctx, prefs):
            return -9999.0

        # raw weighted score
        raw = self._score_single(ctx, prefs)

        # cosine-like normalization
        ctx_norm = math.sqrt(sum(v * v for v in ctx.values()))
        pref_norm = math.sqrt(sum(v * v for v in prefs.values()))
        norm_score = raw / (ctx_norm * pref_norm + 1e-9)

        # apply freshness multiplier
        freshness = self.get_item_freshness(item)
        return norm_score * freshness

    # --------------------------------------------------------
    # FETCH SCORED RESULTS
    # --------------------------------------------------------

    def score_all(self, ctx: dict):
        scored = []
        for item in self.suggestions:
            score = self.calculate_item_score(item, ctx)
            if score >= MIN_SCORE_THRESHOLD:
                scored.append((score, item))
        scored.sort(key=lambda x: x[0], reverse=True)
        return scored

    def get_top_by_subcategory(self, ctx: dict, top_n: int = 3):
        scored = self.score_all(ctx)
        grouped = {}
        for score, item in scored:
            cat = item.get("category", "Unknown")
            sub = item.get("subcategory", "Unknown")
            key = f"{cat} > {sub}"
            grouped.setdefault(key, []).append((score, item))

        for k in grouped:
            grouped[k] = grouped[k][:top_n]

        return grouped

# ============================================================
# 8. SMART SELECTOR — BUNDLES + TIME + EVENTS + HOLIDAYS
# ============================================================

class SmartSelector:
    def __init__(self, suggestion_engine: SuggestionEngine):
        self.engine = suggestion_engine

        # ---------------------------------------------
        # 1) BUNDLE MAP — hero subcategory → sidekicks
        # ---------------------------------------------
        self.BUNDLE_MAP = {
            # --- FOOD & DRINK ---
            "warming_drinks":     ["books", "music", "home_comfort", "relaxation"],
            "cooling_drinks":     ["outdoor_shade", "quick_snack", "accessories_weather"],
            "light_meal":         ["productivity_focus", "creative", "travel_errand"],
            "comfort_meal":       ["movies", "social_community", "home_comfort", "cooling_drinks"],
            "quick_snack":        ["movies", "music", "boosters", "travel_errand"],
            "seasonal_special":   ["social_community", "books", "sensory", "warming_drinks"],

            # --- ACTIVITIES ---
            "outdoor_active":     ["cooling_drinks", "footwear", "protection_items", "accessories_weather"],
            "outdoor_shade":      ["cooling_drinks", "books", "sensory", "accessories_weather"],
            "indoor_cozy":        ["warming_drinks", "home_comfort", "music", "creative"],
            "home_micro":         ["music", "warming_drinks", "sensory", "boosters"],
            "fitness_wellness":   ["cooling_drinks", "comfort_recovery", "energizing"],
            "social_community":   ["quick_snack", "movies", "music"],
            "travel_errand":      ["footwear", "accessories_weather", "music", "quick_snack"],

            # --- MEDIA ---
            "movies":             ["quick_snack", "home_comfort", "comfort_recovery", "warming_drinks"],
            "books":              ["warming_drinks", "sensory", "home_comfort"],
            "music":              ["boosters", "sensory", "creative", "travel_errand"],
            "creative":           ["music", "warming_drinks", "productivity_focus"],

            # --- CLOTHING / COMFORT ---
            "layering_core":      ["travel_errand", "outdoor_active"],
            "footwear":           ["travel_errand", "outdoor_active"],
            "comfort_recovery":   ["relaxation", "movies", "warming_drinks"],
            "home_comfort":       ["movies", "books", "relaxation"],

            # --- MOOD / WELL-BEING ---
            "boosters":           ["quick_snack", "music"],
            "relaxation":         ["home_comfort", "sensory", "warming_drinks"],
            "energizing":         ["outdoor_active", "music", "cooling_drinks"],
            "mental_health_support": ["books", "sensory", "indoor_cozy"],
            "productivity_focus": ["music", "home_micro", "light_meal"],
        }

        # ---------------------------------------------
        # 2) TIME GATES — strict hours by subcategory
        # ---------------------------------------------
        self.TIME_GATES = {
            # Food
            "light_meal":        (6.0, 21.0),
            "comfort_meal":      (12.0, 21.5),
            "quick_snack":       (10.0, 25.0),
            "warming_drinks":    (5.0, 26.0),
            "cooling_drinks":    (10.0, 20.0),

            # Activities
            "outdoor_active":    (6.0, 19.0),
            "outdoor_shade":     (10.0, 17.0),
            "travel_errand":     (9.0, 21.0),
            "social_community":  (10.0, 22.0),

            # Media / Mood
            "movies":            (13.0, 25.0),
            "energizing":        (6.0, 17.0),
            "productivity_focus":(7.0, 19.0),
        }

    # --------------------------------------------------------
    # PHASE 1 — Contextual Permission (time-of-day, weekend, holiday)
    # --------------------------------------------------------
    def get_allowed_subcategories(self, t: dict, is_weekend_like: bool, is_holiday: bool) -> set:
        allowed = set()

        is_morning   = t.get("time_morning", 0) > 0.4 or t.get("time_early_morning", 0) > 0.4
        is_afternoon = t.get("time_afternoon", 0) > 0.4
        is_evening   = t.get("time_evening", 0) > 0.4
        is_night     = t.get("time_night", 0) > 0.4 or t.get("time_late_night", 0) > 0.4

        hour = datetime.datetime.now().hour
        is_lunch_window  = 12 <= hour <= 15
        is_dinner_window = 19 <= hour <= 22

        # --- FOOD ---
        if is_morning:
            allowed.update(["warming_drinks", "light_meal", "seasonal_special"])

        if is_lunch_window:
            allowed.update(["comfort_meal", "cooling_drinks", "seasonal_special"])
            allowed.discard("light_meal")

        if is_afternoon and not is_lunch_window:
            allowed.update(["warming_drinks", "cooling_drinks", "quick_snack", "boosters"])

        if is_dinner_window:
            allowed.update(["comfort_meal", "light_meal", "seasonal_special"])

        if is_night and not is_dinner_window:
            allowed.update(["quick_snack", "warming_drinks"])

        # --- ALWAYS OK SUPPORT CATEGORIES ---
        allowed.update([
            "home_micro", "home_comfort", "music", "sensory",
            "mental_health_support", "layering_core",
            "accessories_weather", "protection_items", "footwear"
        ])

        # --- WEEKEND / HOLIDAY LOGIC ---
        if is_weekend_like:
            # Weekend/holiday: more social, fun, media, some outdoor
            allowed.update(["social_community", "creative", "movies", "books", "travel_errand"])
            if not is_night:
                allowed.update(["outdoor_active", "outdoor_shade", "fitness_wellness"])
            else:
                allowed.update(["relaxation", "comfort_recovery"])
        else:
            # Weekday: productivity first, then evening relaxation/media
            if is_morning or is_afternoon:
                allowed.update(["productivity_focus", "travel_errand", "fitness_wellness"])
            if is_evening or is_night:
                allowed.update(["movies", "books", "relaxation", "creative", "comfort_recovery"])

        # If explicitly a holiday, push more social/media/home, less productivity
        if is_holiday:
            allowed.update([
                "social_community",
                "movies",
                "comfort_meal",
                "home_comfort",
                "books",
                "creative",
                "relaxation",
            ])
            if "productivity_focus" in allowed:
                allowed.remove("productivity_focus")

        return allowed

    # --------------------------------------------------------
    # PHASE 2 — Time Gate Check
    # --------------------------------------------------------
    def is_time_appropriate(self, item: dict, hour: float) -> bool:
        sub = item.get("subcategory", "")
        if sub not in self.TIME_GATES:
            return True

        start, end = self.TIME_GATES[sub]
        check_hour = hour
        if hour < 4.0:  # handle wrap-around for night (0–4 → 24–28)
            check_hour += 24.0

        return start <= check_hour <= end

    # --------------------------------------------------------
    # PHASE 3 — Softmax Weighted Choice
    # --------------------------------------------------------
    def weighted_random_choice(self, scored_items, top_n=1, temp=1.0):
        if not scored_items:
            return []

        max_score = max(x["score"] for x in scored_items)
        weights = [math.exp((x["score"] - max_score) / temp) for x in scored_items]

        chosen = []
        available = list(zip(scored_items, weights))

        for _ in range(min(top_n, len(scored_items))):
            if not available:
                break
            candidates, w = zip(*available)
            selection = random.choices(list(candidates), weights=w, k=1)[0]
            chosen.append(selection)
            # Remove selected to avoid duplicates
            available = [pair for pair in available if pair[0] is not selection]

        return chosen

    # --------------------------------------------------------
    # PHASE 4 — Event-Aware Bundle Biases
    # --------------------------------------------------------
    def apply_event_bundle_biases(self, candidates, context, day_info):
        events_raw = day_info.get("events") or []
        normalized_events = set()
        for e in events_raw:
            key = _normalize_event_key(str(e))
            canon = EVENT_CANONICAL_MAP.get(key)
            if canon:
                normalized_events.add(canon)

        def boost_subcategories(subcats, mul):
            for c in candidates:
                if c["item"].get("subcategory") in subcats:
                    c["score"] *= mul

        def suppress_subcategories(subcats, mul):
            for c in candidates:
                if c["item"].get("subcategory") in subcats:
                    c["score"] *= mul

        # Yalda — cozy, warm, family/home
        if "yalda" in normalized_events:
            boost_subcategories(
                ["indoor_cozy", "home_comfort", "warming_drinks", "books",
                 "comfort_meal", "seasonal_special", "relaxation"],
                1.30
            )
            suppress_subcategories(["outdoor_active", "outdoor_shade"], 0.75)

        # Norooz — fresh, outdoor, social
        if "norooz" in normalized_events:
            boost_subcategories(
                ["outdoor_active", "travel_errand", "seasonal_special",
                 "social_community", "light_meal", "energizing"],
                1.25
            )
            suppress_subcategories(["indoor_cozy", "home_comfort"], 0.9)

        # Sizdah-Bedar — very outdoor
        if "sizdah_bedar" in normalized_events:
            boost_subcategories(
                ["outdoor_active", "travel_errand", "cooling_drinks",
                 "quick_snack", "social_community"],
                1.40
            )
            suppress_subcategories(["indoor_cozy", "home_comfort"], 0.7)

        # Valentine — romantic, cozy, couple
        if "valentine" in normalized_events:
            boost_subcategories(
                ["warming_drinks", "indoor_cozy", "home_comfort",
                 "books", "music", "creative", "light_meal", "relaxation"],
                1.35
            )
            suppress_subcategories(["outdoor_active"], 0.85)

        # Religious Eids — family, food, gatherings
        if any(e in normalized_events for e in ["eid_fitr", "eid_adha", "eid_mabath", "mid_shaban"]):
            boost_subcategories(
                ["comfort_meal", "seasonal_special", "social_community",
                 "home_comfort", "warming_drinks"],
                1.20
            )

        # Mourning days — quieter suggestions
        if any(e in normalized_events for e in ["ashura", "tasua", "arbaeen"]):
            boost_subcategories(
                ["mental_health_support", "books", "indoor_cozy",
                 "home_comfort", "light_meal"],
                1.25
            )
            suppress_subcategories(
                ["movies", "social_community", "boosters", "energizing"],
                0.65
            )

        # Family days — home/family/media
        if any(e in normalized_events for e in ["mother_day", "father_day", "women_day", "men_day"]):
            boost_subcategories(
                ["home_comfort", "indoor_cozy", "warming_drinks",
                 "books", "movies", "light_meal"],
                1.22
            )

        return candidates

    # --------------------------------------------------------
    # MASTER: Generate Bundle
    # --------------------------------------------------------
    def generate_bundle(self, context: dict, day_info: dict):
        now = datetime.datetime.now()
        hour = now.hour + now.minute / 60.0

        # weekend-like from context:
        is_weekend_like = (
            context.get("day_weekend", 0.0) > 0.5
            or context.get("day_holiday", 0.0) > 0.5
            or context.get("day_holiday_eve", 0.0) > 0.5
        )
        is_holiday = bool(day_info.get("is_holiday", False))

        # 1) Allowed subcategories
        allowed = self.get_allowed_subcategories(
            {k: v for k, v in context.items() if k.startswith("time_")},
            is_weekend_like,
            is_holiday,
        )

        # 2) Hero candidates
        candidates = []
        for item in self.engine.suggestions:
            sub = item.get("subcategory", "")
            if sub not in allowed:
                continue
            if not self.is_time_appropriate(item, hour):
                continue

            score = self.engine.calculate_item_score(item, context)
            if score > -0.5:
                candidates.append({"item": item, "score": score})

        # Apply event biases
        candidates = self.apply_event_bundle_biases(candidates, context, day_info)

        heroes = self.weighted_random_choice(candidates, top_n=1, temp=0.8)
        if not heroes:
            return None

        hero = heroes[0]
        result = {"main": hero}

        # 3) Sidekicks
        hero_sub = hero["item"].get("subcategory", "")
        target_subs = self.BUNDLE_MAP.get(hero_sub, [])

        side_candidates = []
        for it in self.engine.suggestions:
            if it is hero["item"]:
                continue
            if it.get("subcategory") not in target_subs:
                continue
            if not self.is_time_appropriate(it, hour):
                continue

            s_score = self.engine.calculate_item_score(it, context)
            if s_score > 0.1:
                side_candidates.append({"item": it, "score": s_score})

        side_candidates = self.apply_event_bundle_biases(side_candidates, context, day_info)

        if side_candidates:
            # Remove duplicates by text
            unique = {}
            for sc in side_candidates:
                txt = sc["item"].get("text", "")
                unique[txt] = sc
            clean = list(unique.values())
            sidekicks = self.weighted_random_choice(clean, top_n=2, temp=1.0)
            result["sidekicks"] = sidekicks

        return result


# ============================================================
# 9. WEATHER FETCHING
# ============================================================

def fetch_weather():
    url = (
        "https://api.open-meteo.com/v1/forecast"
        f"?latitude={LATITUDE}&longitude={LONGITUDE}"
        "&current=temperature_2m,relative_humidity_2m,apparent_temperature,"
        "precipitation,weather_code,cloud_cover,wind_speed_10m"
        "&daily=sunrise,sunset"
        "&timezone=auto"
    )
    print("🌍 Fetching weather from Open-Meteo...")

    try:
        with urllib.request.urlopen(url) as response:
            data = json.loads(response.read().decode())
    except Exception as e:
        print(f"❌ Weather API failed: {e}")
        return None

    sunrise_str = data["daily"]["sunrise"][0].split("T")[1]
    sunset_str  = data["daily"]["sunset"][0].split("T")[1]

    def to_hour(t: str) -> float:
        hh, mm = t.split(":")[:2]
        return int(hh) + int(mm) / 60.0

    return data, to_hour(sunrise_str), to_hour(sunset_str)


# ============================================================
# 10. MAIN — Glue Everything Together
# ============================================================

def main():
    # 1) Weather
    wt = fetch_weather()
    if not wt:
        print("⚠️ Using fallback dummy weather.")
        weather_data = {
            "current": {
                "temperature_2m": 20.0,
                "apparent_temperature": 20.0,
                "relative_humidity_2m": 50.0,
                "weather_code": 0,
                "wind_speed_10m": 5.0,
                "precipitation": 0.0,
                "cloud_cover": 0.0,
            }
        }
        sunrise = 6.0
        sunset  = 18.0
    else:
        weather_data, sunrise, sunset = wt

    # 2) PERSIAN DAY INPUT (from your app)
    # Replace this block with real values from your Persian calendar module.
    day_info = {
        "day_type": "day_workday",    # "day_weekend" / "day_holiday" / "day_holiday_eve"
        "persian_month": 9,          # e.g., Azar
        "persian_day": 10,
        "is_holiday": False,
        "events": ["یلدا"],                # e.g., ["یلدا", "روز مادر"]
    }

    # 3) BUILD CONTEXT
    builder = ContextBuilder()
    ctx_now = builder.build(weather_data, sunrise, sunset, day_info)

    # 3b) PRINT CURRENT TIME / DATE / EVENTS
    now = datetime.datetime.now()
    g_date = now.strftime("%Y-%m-%d")
    g_time = now.strftime("%H:%M")

    print("\n🕒 CURRENT DATE & TIME")
    print("----------------------------------------")
    print(f"Gregorian:  {g_date}  |  {g_time}")
    print(f"Persian:    {day_info['persian_month']:02d}/{day_info['persian_day']:02d}")
    print(f"Day Type:   {day_info['day_type']}")
    print(f"Holiday:    {day_info['is_holiday']}")
    if day_info.get("events"):
        print(f"Events:     {', '.join(day_info['events'])}")
    else:
        print("Events:     None")

    # 3c) SHOW TOP CONTEXT FEATURES
    print("\n📊 Top active context features (NOW):")
    for k, v in sorted(ctx_now.items(), key=lambda x: x[1], reverse=True)[:20]:
        print(f"  {k}: {v:.2f}")

    # 4) LOAD DATA
    engine = SuggestionEngine()
    engine.load_data(DATA_DIR)

    # GLOBAL FRESHNESS RECOVERY
    engine.refresh_all_before_run()

    # 5) GENERATE BUNDLE
    selector = SmartSelector(engine)
    bundle = selector.generate_bundle(ctx_now, day_info)

    if not bundle:
        print("\n⚠️ No recommendation could be generated.")
        return

    main_item = bundle["main"]["item"]
    main_score = bundle["main"]["score"]

    print("\n🎁 SMART BUNDLE RECOMMENDATION")
    print("-" * 80)
    print(f"MAIN [{main_score:.2f}]: {main_item.get('text', 'Unknown')}")

    side_items = bundle.get("sidekicks", [])
    for sk in side_items:
        print(f"  ↳ SIDEKICK [{sk['score']:.2f}]: {sk['item'].get('text', 'Unknown')}")

    # 6) APPLY FRESHNESS DECAY
    items_to_decay = [main_item] + [sk["item"] for sk in side_items]
    engine.apply_decay_for_items(items_to_decay)
    engine.save_freshness_state()

    print("\n✅ Done.")


if __name__ == "__main__":
    main()




